In [84]:
from importlib import reload
reload(metric)

<module 'metric' from 'D:\\device_usage\\metric.py'>

In [85]:
import pandas as pd
from schema import schema
import metric
from metric import *

In [2]:
# Read data
df = pd.read_csv(r'data.csv', dtype=schema, encoding='utf-8')
# Pre-processing
df.drop(['state'], axis=1, inplace=True)
df.fillna(0, inplace=True)

In [3]:
df['plan_c'].value_counts()

10.0      847459
0.0       188760
30.0       63016
6.0            5
2048.0         2
1024.0         2
Name: plan_c, dtype: int64

In [4]:
# Remove outlier  (plan_c == 6)
df = df[df['plan_c']!=6]

In [5]:
# Assign all $0 plan to $10
df_nozeroplan = df.copy()
df_nozeroplan['plan_c'] = df['plan_c'].replace(0.0, 10.0)
df_nozeroplan['plan_s'] = df['plan_s'].replace(0.0, 10.0)

In [6]:
df_nozeroplan['best_plan'] = df_nozeroplan['used'].apply(get_best_plan)

In [7]:
df_nozeroplan['best_plan'].value_counts()

2       451771
5       363756
10      272963
30        8477
100       1436
300        671
500        135
700         21
2048         5
1024         4
Name: best_plan, dtype: int64

In [8]:
df_nozeroplan.head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan
0,898607B4071790000000,3.931,10.0,6.069,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
1,898607B4071790000001,0.000,10.0,10.000,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,2
2,898607B4071790000002,4.877,10.0,5.123,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
3,898607B4071790000003,4.259,10.0,5.741,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
4,898607B4071790000004,1.740,10.0,8.260,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,2


In [9]:
df_nozeroplan['plan_c'].value_counts()

10.0      1036219
30.0        63016
2048.0          2
1024.0          2
Name: plan_c, dtype: int64

Calculate best plan to subscribe to for each device, without considering data pool.

In [10]:
df_nozeroplan['best_plan'] = df_nozeroplan['used'].apply(get_best_plan)

In [11]:
df_nozeroplan.head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan
0,898607B4071790000000,3.931,10.0,6.069,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
1,898607B4071790000001,0.000,10.0,10.000,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,2
2,898607B4071790000002,4.877,10.0,5.123,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
3,898607B4071790000003,4.259,10.0,5.741,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,5
4,898607B4071790000004,1.740,10.0,8.260,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,2,2


In [12]:
df_nozeroplan['best_plan'].value_counts()

2       451771
5       363756
10      272963
30        8477
100       1436
300        671
500        135
700         21
2048         5
1024         4
Name: best_plan, dtype: int64

In [13]:
# Best plan cost
cost_best_plan = dict()
for i in [2,3,4,5]:
    print("Month: {}".format(i))
    best_cost = calculate_cost_with_data_pool(df_nozeroplan[df_nozeroplan['month']==i], col_plan = 'best_plan', verbose=True)
    cost_best_plan.update({i: best_cost})
print(cost_best_plan)

Month: 2
Total devices: 202974
Total cost: 366133.0
Total usage:803452.053999995
   best_plan   count        used  pool_size  remaining_quota  \
0          2   52916   51794.632     105832        54037.368   
1          5  140749  640646.656     703745        63098.344   
2         10    7760   65716.938      77600        11883.062   
3         30    1435   31081.186      43050        11968.814   
4        100      84    5441.666       8400         2958.334   
5        300      20    4088.497       6000         1911.503   
6        500       6    2357.997       3000          642.003   
7        700       4    2324.482       2800          475.518   

   pool_utilization  plan_cost  extra_cost  total_cost  
0          0.489404    52916.0           0     52916.0  
1          0.910339   281498.0           0    281498.0  
2          0.846868    23280.0           0     23280.0  
3          0.721979     7175.0           0      7175.0  
4          0.647817      672.0           0       672.0  


In [14]:
# Actual cost in previous months, using 10 MB for all plan_c == 0 devices
cost_nozeroplan = dict()
for i in [2,3,4,5]:
    best_cost = calculate_cost_with_data_pool(df_nozeroplan[df_nozeroplan['month']==i], verbose=True)
    cost_nozeroplan.update({i: best_cost})
print(cost_nozeroplan)  # beware, this cost assumes all plan_c==0 are using 10 MB plan

Total devices: 202974
Total cost: 631332.0
Total usage:803452.0540000113
   plan_c   count        used  pool_size  remaining_quota  pool_utilization  \
0    10.0  191769  722826.027  1917690.0      1194863.973          0.376925   
1    30.0   11205   80626.027   336150.0       255523.973          0.239851   

   plan_cost  extra_cost  total_cost  
0   575307.0           0    575307.0  
1    56025.0           0     56025.0  
Total devices: 207341
Total cost: 645285.0
Total usage:942648.4099999964
   plan_c   count        used  pool_size  remaining_quota  pool_utilization  \
0    10.0  195710  830002.003  1957100.0      1127097.997          0.424098   
1    30.0   11631  112646.407   348930.0       236283.593          0.322834   

   plan_cost  extra_cost  total_cost  
0   587130.0           0    587130.0  
1    58155.0           0     58155.0  
Total devices: 215835
Total cost: 674167.0
Total usage:981954.5119999921
   plan_c   count        used  pool_size  remaining_quota  pool_utiliza

# Strategy_1
## Rules
1. If a device was activated before this month, use last month's usage as prediction and evaluate this month's plan.
2. If a device was not activated before this month (i.e. first seen this month), use plan_s as this month's plan.

In [15]:
lst_stg_1 = []
cost_strategy_1 = dict()
for mth in [3, 4, 5]:
    df_this_month = df_nozeroplan[df_nozeroplan['month']==mth]
    df_this_month.set_index('iccid', inplace=True)
    df_last_month = df_nozeroplan[df_nozeroplan['month']==mth-1][['iccid','best_plan']]
    df_last_month.rename({'best_plan': 'plan_st_1'}, axis=1, inplace=True)
    df_last_month.set_index('iccid', inplace=True)
    # Get last month's best plan
    df_this_month = df_this_month.join(df_last_month, how='left')
    # Fill plan_s == 0 with 10.0
    df_this_month['plan_s'].replace(0.0, 10.0, inplace=True)
    # Fill in NaN best_plan_lastmonth
    df_this_month['plan_st_1'].fillna(df_this_month['plan_s'], inplace=True)
    lst_stg_1.append(df_this_month)
    #df_this_month.to_csv('df_nonzero_{}.csv'.format(mth))
    # Calculate cost of this month, using strategy 1
    print("Month {}:".format(mth))
    cost_strategy_1.update({mth: calculate_cost_with_data_pool(df_this_month, col_plan='plan_st_1', verbose=True)})
print(cost_strategy_1)

Month 3:
Total devices: 207341
Total cost: 395978.317631997
Total usage:942648.409999997
    plan_st_1   count        used  pool_size  remaining_quota  \
0         2.0   52916   89147.630   105832.0        16684.370   
1         5.0  140749  714625.193   703745.0       -10880.193   
2        10.0    9438   78976.325    94380.0        15403.675   
3        30.0    4121   44771.055   123630.0        78858.945   
4       100.0      84    5248.153     8400.0         3151.847   
5       300.0      20    4928.189     6000.0         1071.811   
6       500.0       6    2723.587     3000.0          276.413   
7       700.0       4    2208.745     2800.0          591.255   
8      2048.0       1      15.853     2048.0         2032.147   
9      3072.0       1       0.000     3072.0         3072.000   
10     4096.0       1       3.680     4096.0         4092.320   

    pool_utilization  plan_cost    extra_cost     total_cost  
0           0.842350    52916.0      0.000000   52916.000000  
1   

In [16]:
# Compare difference
lst_cost = [cost_nozeroplan, cost_strategy_1, cost_best_plan]
df_compare = pd.DataFrame(lst_cost, index=['Paid cost', 'Cost with stg 1', 'Cost with best plan'])

In [17]:
df_compare.head()

,2,3,4,5
Paid cost,631332.0,645285.000000,674167.0,726531.00000
Cost with stg 1,NaN,395978.317632,459697.0,539172.17952
Cost with best plan,366133.0,418069.000000,447816.0,438988.00000


In [18]:
df[df['month']==3].to_csv('df_mar.csv')

In [19]:
df_nozeroplan.columns

Index(['iccid', 'used', 'plan_c', 'left', 'ac_time', 'product', 'company',
       'carrier', 'plan_s', 'month', 'best_plan'],
      dtype='object')

In [20]:
df_nozeroplan['best_plan'] = df_nozeroplan['used'].apply(get_best_plan)

In [21]:
df_nozeroplan['best_plan_fixed_cost'], df_nozeroplan['best_plan_total_cost'], _ = zip(*df_nozeroplan.apply(
        lambda row: compute_cost_with_plan_and_usage(plan=row['best_plan'], usage=row['used']),axis=1))

In [22]:
sum(df_nozeroplan[df_nozeroplan['month']==3]['best_plan_fixed_cost'])

418069.0

In [23]:
df_stg_1_Mar = lst_stg_1[0]

In [24]:
df_stg_1_Mar['best_plan_fixed_cost'], df_stg_1_Mar['best_plan_total_cost'], _ = zip(*df_stg_1_Mar.apply(
        lambda row: compute_cost_with_plan_and_usage(plan=row['best_plan'], usage=row['used']),axis=1))

In [25]:
df_stg_1_Mar['st_1_plan_fixed_cost'], df_stg_1_Mar['st_1_plan_total_cost'], _ = zip(*df_stg_1_Mar.apply(
        lambda row: compute_cost_with_plan_and_usage(plan=row['plan_st_1'], usage=row['used']),axis=1))

In [26]:
sum(df_stg_1_Mar['st_1_plan_fixed_cost'])

384837.0

In [27]:
df_stg_1_Mar[df_stg_1_Mar['best_plan_fixed_cost']>df_stg_1_Mar['st_1_plan_fixed_cost']]

,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan,plan_st_1,best_plan_fixed_cost,best_plan_total_cost,st_1_plan_fixed_cost,st_1_plan_total_cost
iccid,,,,,,,,,,,,,,,
898607B4071790000001,4.914,10.0,5.086,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,5,2.0,2.0,2.000000,1.0,3.983936
898607B4071790000004,6.137,10.0,3.863,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,10,2.0,3.0,3.000000,1.0,5.236288
898607B4071790000006,6.602,10.0,3.398,2018-01-16 15:19:21,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,10,5.0,3.0,3.000000,2.0,3.640448
898607B4071790000011,6.559,10.0,3.441,2018-01-16 15:19:55,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,10,5.0,3.0,3.000000,2.0,3.596416
898607B4071790000012,5.253,10.0,4.747,2018-01-16 15:14:06,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,5,2.0,2.0,2.259072,1.0,4.331072
898607B4071790000034,6.140,10.0,3.860,2018-01-16 15:08:18,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,10,5.0,3.0,3.000000,2.0,3.167360
898607B4071790000037,12.536,10.0,0.000,2018-01-16 15:08:12,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,10,5.0,3.0,3.735440,2.0,9.716864
898607B4071790000052,3.032,10.0,6.968,2018-01-16 15:13:43,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,5,2.0,2.0,2.000000,1.0,2.056768
898607B4071790000054,6.230,10.0,3.770,2018-01-16 15:14:59,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,10,5.0,3.0,3.000000,2.0,3.259520


In [28]:
df_stg_1_Mar['plan_total_diff'] = df_stg_1_Mar['best_plan_total_cost'] - df_stg_1_Mar['st_1_plan_total_cost']
df_stg_1_Mar['plan_fixed_diff'] = df_stg_1_Mar['best_plan_fixed_cost'] - df_stg_1_Mar['st_1_plan_fixed_cost']

In [29]:
sum(df_stg_1_Mar['plan_fixed_diff'])

33232.0

In [30]:
df_stg_1_Mar.head()

,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan,plan_st_1,best_plan_fixed_cost,best_plan_total_cost,st_1_plan_fixed_cost,st_1_plan_total_cost,plan_total_diff,plan_fixed_diff
iccid,,,,,,,,,,,,,,,,,
898607B4071790000000,5.938,10.0,4.062,2018-01-16 15:21:11,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,5,5.0,2.0,2.960512,2.0,2.960512,0.000000,0.0
898607B4071790000001,4.914,10.0,5.086,2018-01-16 15:21:44,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,5,2.0,2.0,2.000000,1.0,3.983936,-1.983936,1.0
898607B4071790000002,5.806,10.0,4.194,2018-01-16 15:19:09,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,5,5.0,2.0,2.825344,2.0,2.825344,0.000000,0.0
898607B4071790000003,5.777,10.0,4.223,2018-02-09 11:10:04,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,5,5.0,2.0,2.795648,2.0,2.795648,0.000000,0.0
898607B4071790000004,6.137,10.0,3.863,2018-01-16 15:21:28,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3,10,2.0,3.0,3.000000,1.0,5.236288,-2.236288,2.0


In [31]:
sum(df_stg_1_Mar['used']), len(df_stg_1_Mar)*2

(942648.4100000005, 414682)

In [32]:
dollar_per_mb = dict()
for k, v in fc.items():
    dollar_per_mb.update({k: v/k})
sorted_by_value = sorted(dollar_per_mb.items(), key=lambda kv: kv[1], reverse=True)

In [33]:
sorted_by_value

[(2, 0.5),
 (5, 0.4),
 (10, 0.3),
 (30, 0.16666666666666666),
 (100, 0.08),
 (300, 0.05333333333333334),
 (500, 0.048),
 (700, 0.045714285714285714),
 (1024, 0.0390625),
 (2048, 0.029296875),
 (3072, 0.026041666666666668),
 (4096, 0.0244140625)]

In [43]:
p = [x[0] for x in sorted_by_value]

In [44]:
for x in p[1:]:
    print(x)

5
10
30
100
300
500
700
1024
2048
3072
4096


In [34]:
dict_pool = df_stg_1_Mar.groupby('best_plan')['best_plan'].count().to_dict()

In [39]:
dict_pool

{2: 60554,
 5: 88960,
 10: 55510,
 30: 2081,
 100: 173,
 300: 45,
 500: 8,
 700: 7,
 1024: 2,
 2048: 1}

In [37]:
available_pool = dict()
for k,v in dict_pool.items():
    available_pool[k] = k*v

In [38]:
available_pool

{2: 121108,
 5: 444800,
 10: 555100,
 30: 62430,
 100: 17300,
 300: 13500,
 500: 4000,
 700: 4900,
 1024: 2048,
 2048: 2048}

In [45]:
sorted(fc.keys(), reverse=False)

[2, 5, 10, 30, 100, 300, 500, 700, 1024, 2048, 3072, 4096]

In [62]:
for row in df_stg_1_Mar.head().itertuples():
    print(df_stg_1_Mar.row.index)
    print(getattr(row, "index"))

AttributeError: 'DataFrame' object has no attribute 'row'

In [63]:
bins = iter([1,2,3,4,5])

In [69]:
cur_bin = next(bins)
print(cur_bin)

StopIteration: 

In [73]:
df_stg_1_Mar.loc['898607B4071790000000','used']

5.938

In [86]:
df_res, total_cost = compute_best_global_plan_with_pool(df_nozeroplan[df_nozeroplan['month']==3])

D:\device_usage\metric.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\device_usage\metric.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cur_bin = next(bins)
D:\device_usage\metric.py:181: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  total_cost = calculate_cost_with_data_pool(df_result, col_plan='best_global_plan', col_used='used')


In [87]:
lst_best = []
for mth in [2,3,4,5]:
    df_res, total_cost = compute_best_global_plan_with_pool(df_nozeroplan[df_nozeroplan['month']==mth])
    lst_best.append(df_res)
    print("Month: {}  Cost:{}".format(mth, total_cost))

D:\device_usage\metric.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\device_usage\metric.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cur_bin = next(bins)
D:\device_usage\metric.py:181: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  total_cost = calculate_cost_with_data_pool(df_result, col_plan='best_global_plan', col_used='used')


Month: 2  Cost:0.0
Month: 3  Cost:0.0
Month: 4  Cost:0.0
Month: 5  Cost:0.0


In [88]:
lst_best[1].head()

,iccid,used,plan_c,left,ac_time,product,company,carrier,plan_s,month,best_plan,best_plan_fixed_cost,best_plan_total_cost,best_local_plan,best_global_plan
202988,898607B4071790000019,0.0,10.0,10.0,2018-01-16 15:06:02,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3.0,2.0,1.0,1.0,2.0,NaN
202992,898607B4071790000023,0.0,10.0,10.0,2018-01-16 15:20:32,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3.0,2.0,1.0,1.0,2.0,NaN
202995,898607B4071790000026,0.0,10.0,10.0,2018-01-16 15:15:23,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3.0,2.0,1.0,1.0,2.0,NaN
202999,898607B4071790000030,0.0,10.0,10.0,2018-01-16 15:19:50,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3.0,2.0,1.0,1.0,2.0,NaN
203000,898607B4071790000031,0.0,10.0,10.0,2018-01-16 15:20:07,SPB132,挚享科技（上海）有限公司,北京华虹,10.0,3.0,2.0,1.0,1.0,2.0,NaN
